#Load libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchaudio
!pip install pydub
!pip install spotipy

     |████████████████████████████████| 1.9 MB 4.0 MB/s 


In [ ]:
import torch
from torch import nn
from torch._C import device
from torch.nn.modules.activation import ReLU
from torch.nn.modules.linear import Linear
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torchvision
from sklearn import preprocessing
from torch import Tensor
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
import torchaudio as ta
import librosa
import numpy as np
import scipy
from scipy import misc
import glob
import random
from PIL import Image
import os
from pydub import AudioSegment
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from collections import Counter
import shutil
from urllib.request import urlretrieve
from spotipy.oauth2 import SpotifyClientCredentials

## Checking GPU availability

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device} device")

Using cuda device


## Create Spotify Dataset

In [ ]:
def spotify_login(cid, secret):
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

cid = "User_ID"
secret = "Secret_PWD"

spotify = spotify_login(cid, secret)

In [ ]:
playlists = {'Happy' : ["37i9dQZF1DXdPec7aLTmlC", "1h90L3LP8kAJ7KGjCV2Xfd"],
             'Sad' : ["37i9dQZF1DX7qK8ma5wgG1", "4kPr5JnmXhy9OxewtmylVI"],
             'Energetic' : ["2lmcuXNkjYOoQeXvwqvvFT", "37i9dQZF1DWT6anPZiHuxz"],
             'Dark' : ["37i9dQZF1DX2pSTOxoPbx9", "4Nv06dxNVcYcL88w8WbZVY"],
             'Relaxing' : ["37i9dQZF1DWTC99MCpbjP8", "7jM62qbx0nLl82afAgU3kb"]}

In [ ]:
def write_songs(spotify, directory, id, mood):
  playlist_tracks = spotify.playlist_tracks(id)
  preview_urls = []
  counter = 0
  for track in playlist_tracks['items']:
    try:
      if track['track']['preview_url'] is None:
        counter = counter + 1
        continue
      preview_urls.append(track['track']['preview_url'])
    except:
      counter = counter + 1
      continue
  for i in range(len(preview_urls)):
    urlretrieve(preview_urls[i], "{}/{}{}{}".format(directory, mood, i+1, ".mp3"))
  print(f"number of skipped song in current playlist: {counter}")
  return

In [ ]:
def get_songs(spotify, playlists, directory):
  for mood, ids in playlists.items():
      print(mood)
      for id in ids:
          write_songs(spotify, directory + "/" + mood, id, mood)
  return

In [ ]:
get_songs(spotify, playlists, "/content/drive/MyDrive/Colab Notebooks/music_project/team_project/Spotify")

Happy
number of skipped song in current playlist: 37
number of skipped song in current playlist: 3
Sad
number of skipped song in current playlist: 25
number of skipped song in current playlist: 48
Energetic
number of skipped song in current playlist: 51
number of skipped song in current playlist: 9
Dark
number of skipped song in current playlist: 34
number of skipped song in current playlist: 10
Relaxing


In [ ]:
moods = 'Dark Energetic Happy Relaxing Sad'
moods = moods.split()

## Building Folders


In [ ]:
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio3sec')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio5sec')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio10sec')

os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio3sec/train')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio3sec/test')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio3sec/val')

os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio5sec/train')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio5sec/test')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio5sec/val')

os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio10sec/train')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio10sec/test')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio10sec/val')

'''os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/test')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/train')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/val')'''

for m in moods:

  path_train = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio3sec/train',f'{m}')
  path_test = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio3sec/test',f'{m}')
  path_val = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio3sec/val',f'{m}')
  os. makedirs(path_train)
  os. makedirs(path_test)
  os. makedirs(path_val)

  path_train = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio5sec/train',f'{m}')
  path_test = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio5sec/test',f'{m}')
  path_val = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio5sec/val',f'{m}')
  os. makedirs(path_train)
  os. makedirs(path_test)
  os. makedirs(path_val)

  path_train = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio10sec/train',f'{m}')
  path_test = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio10sec/test',f'{m}')
  path_val = os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio10sec/val',f'{m}')
  os. makedirs(path_train)
  os. makedirs(path_test)
  os. makedirs(path_val)

##Creating Train/Test/Val Set and Mel-Spectrograms

In [ ]:
#select the segmentation window (1, 3, 5, 10) seconds
segment = 3

In [ ]:
directory = '/content/drive/MyDrive/Colab Notebooks/music_project/team_project/Spotify'

i=0
for m in moods:
  print(m)
  j=0
  
  filenames = os.listdir(os.path.join(directory,f"{m}"))

  train_files, test_files = train_test_split(filenames, test_size=0.1, random_state=42)
  test_files, val_file = train_test_split(test_files, test_size=0.2, random_state=42)
  for f in test_files:
    j = j+1
    song  =  os.path.join(f'/content/drive/MyDrive/Colab Notebooks/music_project/team_project/Spotify/{m}',f'{f}')
    for w in range(0,30//segment):
      i = i+1
      t1 = segment*(w)*1000
      t2 = segment*(w+1)*1000
      path = '/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio'+str(segment)+'sec/test/'+m+'/'+f+'_'+str(j)+str(w)+'.mp3'
      newAudio = AudioSegment.from_mp3(song)
      new = newAudio[t1:t2]
      new.export(path, format="mp3")

  i=0
  j=0
  for f in train_files:
    j = j+1
    song  =  os.path.join(f'/content/drive/MyDrive/Colab Notebooks/music_project/team_project/Spotify/{m}',f'{f}')
    for w in range(0,30//segment):
      i = i+1
      t1 = segment*(w)*1000
      t2 = segment*(w+1)*1000
      path = '/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio'+str(segment)+'sec/train/'+m+'/'+f+'_'+str(j)+str(w)+'.mp3'
      newAudio = AudioSegment.from_mp3(song)
      new = newAudio[t1:t2]
      new.export(path, format="mp3")

  i=0
  j=0
  for f in val_file:
    j+=1
    song  =  os.path.join(f'/content/drive/MyDrive/Colab Notebooks/music_project/team_project/Spotify/{m}',f'{f}')
    for w in range(0,30//segment):
        i = i+1
        t1 = segment*(w)*1000
        t2 = segment*(w+1)*1000
        path = '/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio'+str(segment)+'sec/val/'+m+'/'+f+'_'+str(j)+str(w)+'.mp3'
        newAudio = AudioSegment.from_mp3(song)
        new = newAudio[t1:t2]
        new.export(path, format="mp3")


Dark
54 4 2
Dark9.mp3 Dark31.mp3 Dark35.mp3
Energetic
44 4 1
Energetic21.mp3 Energetic12.mp3 Energetic40.mp3
Happy
87 8 2
Happy77.mp3 Happy45.mp3 Happy47.mp3
Relaxing
79 7 2
Relaxing17.mp3 Relaxing16.mp3 Relaxing55.mp3
Sad
46 4 2
Sad18.mp3 Sad6.mp3 Sad20.mp3


### TRAIN SPLIT

In [ ]:
X_train = []
Y_train = []
X_train_songs = []

sr = 22050
hop_length = 512
n_fft = 2048

for m in moods:
  j = 0
  print(m)
  for filename in os.listdir(os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio'+str(segment)+'sec/train',f"{m}")):
    song  =  os.path.join(f'/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio{segment}sec/train/{m}',f'{filename}')
    j = j+1
    
    y, sr = ta.load(song)
    S = ta.transforms.MelSpectrogram(sample_rate=sr, hop_length=hop_length, n_fft=n_fft)(y) #(N, channel\ num mels\ time)
    log_S = ta.transforms.AmplitudeToDB(stype='magnitude')(S)

    Y_train.append(m)
    X_train.append(log_S.to(device))
    fields = song.split('/')
    song_name = str(song.split('.')[0])
    X_train_songs.append(song_name)


Dark
Energetic
Happy
Relaxing
Sad


In [ ]:
len(X_train), len(X_train_songs)

(3100, 3100)

### TEST SPLIT

In [ ]:
X_test = []
X_test_songs = []
Y_test = []

sr = 22050
hop_length = 512
n_fft = 2048

for m in moods:
  j = 0
  print(m)
  for filename in os.listdir(os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio'+str(segment)+'sec/test',f"{m}")):
    song  =  os.path.join(f'/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio{segment}sec/test/{m}',f'{filename}')
    j = j+1
    
    y, sr = ta.load(song)
    S = ta.transforms.MelSpectrogram(sample_rate=sr, hop_length=hop_length, n_fft=n_fft)(y) #(N, channel\ num mels\ time)
    log_S = ta.transforms.AmplitudeToDB(stype='magnitude')(S)

    Y_test.append(m)
    X_test.append(log_S.to(device))

    fields = song.split('/')
    song_name = str(song.split('.')[0])
    X_test_songs.append(song_name)


Dark
Energetic
Happy
Relaxing
Sad


### VAL SPLIT

In [ ]:
X_val = []
X_val_songs = []
Y_val = []

sr = 22050
hop_length = 512
n_fft = 2048

for m in moods:
  j = 0
  print(m)
  for filename in os.listdir(os.path.join('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio'+str(segment)+'sec/val',f"{m}")):
    song  =  os.path.join(f'/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood/audio{segment}sec/val/{m}',f'{filename}')
    j = j+1
    
    y, sr = ta.load(song)
    S = ta.transforms.MelSpectrogram(sample_rate=sr, hop_length=hop_length, n_fft=n_fft)(y) #(N, channel\ num mels\ time)
    log_S = ta.transforms.AmplitudeToDB(stype='magnitude')(S)

    Y_val.append(m)
    X_val.append(log_S.to(device))

    fields = song.split('/')
    song_name = str(song.split('.')[0])
    X_val_songs.append(song_name)


Dark
Energetic
Happy
Relaxing
Sad


## Shuffling

In [ ]:
train_array = np.array(X_train)
label_train_array = np.array(Y_train)
song_train_array = np.array(X_train_songs)

test_array = np.array(X_test)
label_test_array = np.array(Y_test)
song_test_array = np.array(X_test_songs)

val_array = np.array(X_val)
label_val_array = np.array(Y_val)
song_val_array = np.array(X_val_songs)

In [ ]:
shuffled_train = np.c_[train_array.reshape(len(train_array), -1), label_train_array.reshape(len(label_train_array), -1), song_train_array.reshape(len(song_train_array), -1)]
np.random.shuffle(shuffled_train)

shuffled_test = np.c_[test_array.reshape(len(test_array), -1), label_test_array.reshape(len(label_test_array), -1), song_test_array.reshape(len(song_test_array), -1)]
np.random.shuffle(shuffled_test)

shuffled_val = np.c_[val_array.reshape(len(val_array), -1), label_val_array.reshape(len(label_val_array), -1), song_val_array.reshape(len(song_val_array), -1)]
np.random.shuffle(shuffled_val)

## Labels Encoder

In [ ]:
Y_train_shuffled = np.array([shuffled_train[i][1] for i in range(len(shuffled_train))])
Y_test_shuffled = np.array([shuffled_test[i][1] for i in range(len(shuffled_test))])
Y_val_shuffled = np.array([shuffled_val[i][1] for i in range(len(shuffled_val))])

In [ ]:
def encode_labels(Y, le=None, enc=None):
    """Encodes target variables into numbers and then one hot encodings"""

    # initialize encoders
    N = Y.shape[0]
    
    # Encode the labels
    if le is None:
        le = preprocessing.LabelEncoder()
        Y_le = le.fit_transform(Y).reshape(N, 1)
    else:
        Y_le = le.transform(Y).reshape(N, 1)        
        
        # convert into one hot encoding
        
    if enc is None:
        enc = preprocessing.OneHotEncoder()
        Y_enc = enc.fit_transform(Y_le).toarray()
    else:
        Y_enc = enc.transform(Y_le).toarray()

    # return encoders to re-use on other data
    return Y_enc, le, enc

In [ ]:
# Encode the target vectors into one-hot encoded vectors
Y_train, le, enc = encode_labels(Y_train_shuffled)
Y_test, le, enc = encode_labels(Y_test_shuffled, le, enc)
Y_val, le, enc = encode_labels(Y_val_shuffled, le, enc)

## Padding

In [ ]:
X_train_shuffled = np.array([shuffled_train[i][0] for i in range(len(shuffled_train))])
X_test_shuffled = np.array([shuffled_test[i][0] for i in range(len(shuffled_test))])
X_val_shuffled = np.array([shuffled_val[i][0] for i in range(len(shuffled_val))])

In [ ]:
S_train_shuffled = np.array([shuffled_train[i][2] for i in range(len(shuffled_train))])
S_test_shuffled = np.array([shuffled_test[i][2] for i in range(len(shuffled_test))])
S_val_shuffled = np.array([shuffled_val[i][2] for i in range(len(shuffled_val))])

In [ ]:
import torch.nn.functional as F

i=0
for el in X_train_shuffled:
  if el.size()[2]!=X_train_shuffled[0].size()[2]:
    last_dim_padding = (0, X_train_shuffled[0].size()[2] - el.size()[2])
    X_train_shuffled[i] = F.pad(el, last_dim_padding)
    print(i)
  i+=1

In [ ]:
i=0
for el in X_test_shuffled:
  if el.size()[2]!=X_test_shuffled[0].size()[2]:
    last_dim_padding = (0, X_test_shuffled[0].size()[2] - el.size()[2])
    X_test_shuffled[i] = F.pad(el, last_dim_padding)
    print(i)
  i+=1

In [ ]:
i=0
for el in X_val_shuffled:
  if el.size()[2]!=X_val_shuffled[0].size()[2]:
    last_dim_padding = (0, X_val_shuffled[0].size()[2] - el.size()[2])
    X_val_shuffled[i] = F.pad(el, last_dim_padding)
    print(i)
  i+=1

### Reshape data as 2d convolutional tensor shape

In [ ]:
torch.device(device)
    
X_train = torch.stack(list(X_train_shuffled)).to(device)
print("X_stacked train size:", X_train.size())
X_test = torch.stack(list(X_test_shuffled)).to(device)
print("X_stacked test size:", X_test.size())
X_val = torch.stack(list(X_val_shuffled)).to(device)
print("X_stacked val size:", X_val.size())

X_stacked train size: torch.Size([3100, 2, 128, 261])
X_stacked test size: torch.Size([270, 2, 128, 261])
X_stacked val size: torch.Size([90, 2, 128, 261])


In [ ]:
if X_train.size()[1] > 1:
      X_train = torch.unsqueeze(torch.mean(X_train, dim=1), dim=1)
if X_test.size()[1] > 1:
      X_test = torch.unsqueeze(torch.mean(X_test, dim=1), dim=1)
if X_val.size()[1] > 1:
      X_val = torch.unsqueeze(torch.mean(X_val, dim=1), dim=1)

In [ ]:
Y_train = torch.tensor(Y_train, dtype=torch.long).to(device)
Y_test = torch.tensor(Y_test, dtype=torch.long).to(device)
Y_val = torch.tensor(Y_val, dtype=torch.long).to(device)

#Hyperparameters

In [ ]:
BATCH_SIZE = 16
EPOCHS = 200
patience_early_stopping = 15
delta_early_stopping = 0.01
LEARNING_RATE = 0.001
nb_classes = 5

#Dataloader

In [ ]:
def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader

##Creating Dataloader

In [ ]:
train_data = TensorDataset(X_train, Y_train)
train_dataloader = create_data_loader(train_data, BATCH_SIZE)

test_data = TensorDataset(X_test, Y_test)
test_dataloader = create_data_loader(test_data, BATCH_SIZE)

val_data = TensorDataset(X_val, Y_val)
val_dataloader = create_data_loader(val_data, BATCH_SIZE)

#Training

## Model Architecture

In [ ]:
class CRNN(nn.Module):

  def __init__(self, n_classes):
        super().__init__()
        self.norm = nn.BatchNorm2d(128)
        self.conv_layers = nn.Sequential(
            

            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ELU(),
            nn.BatchNorm2d(64),
            nn.AvgPool2d((2,2)),
            nn.Dropout(0.3),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ELU(),
            nn.BatchNorm2d(128),
            nn.AvgPool2d((4,2)),
            nn.Dropout(0.3),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ELU(),
            nn.BatchNorm2d(128),
            nn.AvgPool2d((4,2)),
            nn.Dropout(0.3),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ELU(),
            nn.BatchNorm2d(128),
            nn.AvgPool2d((4,2)),
            nn.Dropout(0.3)
        )


        self.GRU1 = nn.GRU(128, 32)
        self.GRU2 = nn.GRU(32, 32)
        
        self.out_layers = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(32, n_classes),
            #nn.Softmax(dim=1)
        )

  def forward(self, input_data):
        x = input_data.transpose(2, 1)
        x = self.norm(x)
        x = x.transpose(2, 1)

        x = self.conv_layers(x)
        x = torch.reshape(x, (x.size()[3], x.size()[0], x.size()[2]*x.size()[1]))

        x, _ = self.GRU1(x)
        x, _ = self.GRU2(x)        
        
        x = torch.reshape(x, (x.size()[1], x.size()[0], x.size()[2])) # (16, 56, 32)
        x = x[:, -1, :] # (16, 1, 32)

        predictions = self.out_layers(x)
        return predictions

## Trainer Section

In [ ]:
def format_results(preds, trues):

  predsBS = []
  truesBS = []

  for i in range(len(preds)-1):
    for j in range(preds[0].size()[0]):
      predsBS.append(torch.argmax(preds[i][j]).item())
      truesBS.append(torch.argmax(trues[i][j]).item())
  for j in range(preds[len(preds)-1].size()[0]):
    predsBS.append(torch.argmax(preds[len(preds)-1][j]).item())
    truesBS.append(torch.argmax(trues[len(preds)-1][j]).item())

  return np.array(predsBS), np.array(truesBS)

In [ ]:
def get_likely_index(tensor):
    return tensor.argmax(dim=-1)

def number_of_correct(pred, target):
    return pred.squeeze().eq(target).sum().item()

In [ ]:
def train_one_epoch(model, data_loader, loss_fn, optimizer, device, val):

  if val==1:

    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0


    for inputs, targets in data_loader:
      with torch.no_grad():
          inputs, targets = inputs.to(device), targets.to(device)
          
          targets_i = []
          j = 0

          predictions = model(inputs)
          pred = get_likely_index(predictions)

          for j in range (targets.size()[0]):
            targets_i.append(torch.argmax(targets[j]).item())
          
          targets_t = torch.LongTensor(targets_i).to(device)

          loss = loss_fn(predictions, targets_t)

          val_loss += loss

          val_steps += 1

          correct += number_of_correct(pred, targets_t)

          total += targets.size(0)

      accuracy = correct / total
      loss_epoch = val_loss / val_steps
      print(f"Loss: {loss_epoch}")
      print(f"Accuracy: {accuracy}")

      return loss_epoch, accuracy

  else:
    for inputs, targets in data_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        total = 0
        correct = 0
        targets_i = []
        j = 0

        predictions = model(inputs)
        pred = get_likely_index(predictions)

        for j in range (targets.size()[0]):
          targets_i.append(torch.argmax(targets[j]).item())
        
        targets_t = torch.LongTensor(targets_i).to(device)

        loss = loss_fn(predictions, targets_t)

        correct += number_of_correct(pred, targets_t)
        total += targets.size(0)

        # backpropagate loss and update weights
        optimizer.zero_grad() # at every iteration we calc gradients. these gradients 
                              # are saved and for this reason we want to reset them
        loss.backward()
        optimizer.step()

    accuracy = correct / total

    print(f"Loss: {loss.item()}")
    print(f"Accuracy: {accuracy}")

    return loss.item(), accuracy


def train(model, train_dataloader, val_dataloader, loss_fn, optimizer, device, epochs, patience, delta):

  train_losses = []
  val_losses = []
  train_accuracies = []
  val_accuracies = []
  epochs_no_improve = 0
  min_loss = None

  val = 0
  print('Starting Training:')
  for i in range(epochs):
      print((f"Epoch {i+1}"))
      loss_epoch, acc_epoch = train_one_epoch(model, train_dataloader, loss_fn, optimizer, device, val)
      train_losses.append(loss_epoch)
      train_accuracies.append(acc_epoch)
      print("----------------------------------------")

      if (min_loss is None or loss_epoch <= min_loss-delta):
        min_loss = loss_epoch
        epochs_no_improve = 0

      else:
        epochs_no_improve = epochs_no_improve + 1

      if epochs_no_improve > patience:
        print("early stopped. Current loss: " + str(loss_epoch) + "\t min loss: " + str(min_loss) + "\t epochs: " + str(i))
        break

  val = 1
  print('Starting Validation:')
  for j in range(i+1):
      print((f"Val Epoch {j+1}"))
      loss_epoch, acc_epoch = train_one_epoch(model, val_dataloader, loss_fn, optimizer, device, val)
      val_losses.append(loss_epoch)
      val_accuracies.append(acc_epoch)
      print("----------------------------------------")


  print("Training is done")
  return train_losses, val_losses, train_accuracies, val_accuracies

##Train model

In [ ]:
CRNN = CRNN(nb_classes).to(device=device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CRNN.parameters(), lr=LEARNING_RATE, weight_decay=0.0005)
#optimizer = torch.optim.SGD(CRNN.parameters(), lr=LEARNING_RATE, momentum=0.9)

train_losses, val_losses, train_accuracies, val_accuracies = train(CRNN, train_dataloader, val_dataloader, loss_fn, optimizer, device, EPOCHS, patience_early_stopping, delta_early_stopping)

torch.save(CRNN.state_dict(), "/content/drive/MyDrive/Colab Notebooks/music_project/team_project/MOOD_x.pth")
print("Model trained and stored at MOOD_x.pth")

# Testing

In [ ]:
from collections import defaultdict
from sklearn.utils import shuffle

def predict(model, X, Y, S):
  
    model.eval()
    with torch.no_grad():
      
        prediction_l = []
        labels_l = []

        songs = np.unique(S)
        
        for song in songs:
          print(song)
          X_song = X[S==song]
          Y_song = Y[S==song]

          predictions = model(X_song)
          class_prediction = [torch.argmax(pred).item() for pred in predictions]
          print(class_prediction)
          actual_prediction = [torch.argmax(label).item() for label in Y_song]

          p = defaultdict(int)
          t = defaultdict(int)

          for i in class_prediction:
              p[i] += 1
          for i in actual_prediction:
              t[i] += 1
          
          pred_song = max(p.items(), key=lambda x: x[1])[0]
          actual_song = max(t.items(), key=lambda x: x[1])[0]

          prediction_l.append(pred_song)
          labels_l.append(actual_song)


    return prediction_l, labels_l

In [ ]:
preds, trues = predict(CRNN, X_test, Y_test, S_test_shuffled)

##Performance Evaluation

In [ ]:
acc = accuracy_score(trues, preds)
f1 = f1_score(trues, preds, average='weighted')
cr = classification_report(trues, preds)
print("accuracy:", acc)
print("f1 score:", f1)
print("classification report:")
print(cr)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training Loss")
plt.plot(train_losses,label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Deleting slicing folder

In [ ]:
shutil.rmtree('/content/drive/MyDrive/Colab Notebooks/music_project/team_project/mood', ignore_errors=True)

#TESTING ON NEW INPUTS

In [ ]:
def load_audio_file_segmented(songpath, segment):

  sound = None
  song_tensor, _= ta.load(songpath)

  song_name = songpath.split('/')[-1].split('.')[0]
  root = '/content/drive/MyDrive/Colab Notebooks/music_project/team_project/song_to_try/'
  os.makedirs(root+'slices')

  if songpath.endswith('.mp3') or songpath.endswith('.MP3'):
      for w in range(0, ((song_tensor.size()[1]//44100)//segment) ):
        t1 = segment*(w)*1000
        t2 = segment*(w+1)*1000
        newAudio = AudioSegment.from_mp3(songpath)
        sound = newAudio[t1:t2]
        sound.export(f'/content/drive/MyDrive/Colab Notebooks/music_project/team_project/song_to_try/slices/{song_name+str(w)}.mp3', format="mp3")
      newpath = root+'slices/'

  elif songpath.endswith('.wav') or songpath.endswith('.WAV'):
      for w in range(0,((song_tensor.size()[1]//22050)//segment)):
        t1 = segment*(w)*1000
        t2 = segment*(w+1)*1000
        newAudio = AudioSegment.from_wav(songpath)
        sound = newAudio[t1:t2]
        sound.export(f'/content/drive/MyDrive/Colab Notebooks/music_project/team_project/song_to_try/slices/{song_name+str(w)}.wav', format="wav")
      newpath = root+'slices/' 
    
  return newpath

def get_data(songpath):
  hop_length = 512
  n_fft = 2048
 
  sound_tensor = []

  for _,_,filenames in os.walk(songpath):
    for f in filenames:
      song = os.path.join(songpath,f'{f}')
      y, sr = ta.load(song)
      S = ta.transforms.MelSpectrogram(sample_rate=sr, hop_length=hop_length, n_fft=n_fft)(y) #(N, channel\ num mels\ time)
      sliced_sound = ta.transforms.AmplitudeToDB(stype='magnitude')(S)
      
      if sliced_sound.size()[0] > 1:
          sliced_sound = torch.unsqueeze(torch.mean(sliced_sound, dim=0), dim=0)

      sound_tensor.append(sliced_sound)
  
  sound_tensor = torch.stack(sound_tensor).to(device)
  return sound_tensor

def predict_song(songpath, segmentSize, model, n_classes, le):
  newpath = load_audio_file_segmented(songpath, segmentSize)
  sound_tensor = get_data(newpath)
  shutil.rmtree(newpath, ignore_errors=True)

  model.eval()
  with torch.no_grad():
    class_prediction = []

    predictions = model(sound_tensor)
    class_prediction = [torch.argmax(pred).item() for pred in predictions]

    p = defaultdict(int)
    for i in class_prediction:
      p[i] += 1
            
    pred_song = max(p.items(), key=lambda x: x[1])[0]
    c = Counter(class_prediction)
    outcomes = [(le.inverse_transform([i])[0], float('%.3f'%(c[i] / len(class_prediction) * 100.0))) for i, count in c.most_common()]
    print(outcomes)

  return outcomes

In [ ]:
CRNN = CRNN(5).to(device=device)
PATH = '/content/drive/MyDrive/Colab Notebooks/music_project/team_project/MOOD_BEST.pth'
CRNN.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
moods = 'Dark Energetic Happy Relaxing Sad'
moods = moods.split()
le = preprocessing.LabelEncoder()
le.fit_transform(moods)

In [ ]:
songpath = '/content/drive/MyDrive/Colab Notebooks/music_project/team_project/song_to_try/reminder.mp3'

In [ ]:
pred = predict_song(songpath,3,CRNN,5,le)

## Multi-Classification visualization

In [ ]:
preds_cut = pred[0:4]

others_sum = round(100-(pred[0][1]+pred[1][1]+pred[2][1]), 4)
others = ('others', others_sum)
preds_cut.pop()
preds_cut.append(others)
preds_cut

In [ ]:
import seaborn as sns

plt.rcParams["figure.figsize"] = [7, 4.5]
plt.rcParams["figure.autolayout"] = True
ax = sns.barplot(x=[p[0] for p in preds_cut], y=[float(p[1]) for p in preds_cut], palette='PuBuGn_r')
patches = ax.patches
for i in range(len(patches)):
   x = patches[i].get_x() + patches[i].get_width()/2
   y = patches[i].get_height()+.01
   ax.annotate('{:.2f}%'.format(preds_cut[i][1]), (x, y), ha='center')
plt.show()

### PLOTTING SPECTROGRAMS

In [ ]:
def plot_specgram(waveform, sample_rate, title, xlim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
  figure.suptitle(title)
  plt.show(block=False)

In [ ]:
song = '/content/drive/MyDrive/Colab Notebooks/music_project/team_project/song_to_try/715.mp3'
title = 'black street'
y, sr = ta.load(song)
if y.size()[0] > 1:
  y = torch.unsqueeze(torch.mean(y, dim=0), dim=0)

print(f"Shape of spectrogram: {y.size()} with sample rate {sr}")
plot_specgram(y,sr,title)